# Bulk viscosity and boundary layers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from tqdm.auto import tqdm

In [ ]:
full_width = (7.68, 4.8)
half_width = (3.84, 4.8)
half_double = (7.68, 9.6)
plt.rcParams['figure.figsize'] = half_width # 50% of the screen, vertical, assuming a 1x2 plot
plt.rcParams['font.size'] = 10

# plt.rcParams.update({'font.size': 16,
#                      'lines.markersize': 12,
#                      'lines.markeredgewidth': 2})
save_plots = False

In [ ]:
%matplotlib notebook

This is like the boundary layers notebook, but with the equations modified to the relativistic case (minor tweak), and figures plotted in talk format.

## Bulk viscosity

Take the relativistic Euler equations in convective form. Assume that we are looking on a sufficiently small scale so that gradients of the pressure are negligible and the divergence of the velocity is constant. Include the effects of a reaction term. Then the equations of motion become
$$
\begin{aligned}
D_t \rho &= - \rho \theta \\
D_t e &= -(e + p(\rho, e, Y)) \theta \\
D_t Y &= -\epsilon^{-1} \Gamma(\rho, e, Y).
\end{aligned}
$$
Here $\theta = \nabla \cdot v$ is the (constant) expansion and $\epsilon \ll 1$ is the reaction timescale, whilst $D_t$ is the convective derivative.

To simplify the system we linearize the reaction rate $\Gamma$ as $\Gamma = A(\rho, e) Y - B(\rho, e)$ and assume that $B / A > 0$ so there is a stable equilibrium solution.

The simplest multiscale calculation makes the assumption that
$$
Y = \sum_{n=0} \epsilon^n Y_n.
$$
Substituting into the equation of motion for the species fraction,
$$
D_t Y = -\epsilon^{-1} \left( A Y - B \right),
$$
and gathering like powers in $\epsilon$, we find
$$
Y_0 = \frac{B}{A} = Y_{\text{eq}}(\rho, e)
$$
and
$$
Y_1 = -A^{-1} D_t Y_0 = A^{-1} \left( \rho \partial_\rho Y_{\text{eq}}+ (e + p_{\text{eq}}) \partial_e Y_{\text{eq}} \right) \theta.
$$
From this we can expand the pressure in powers of $\epsilon$ about equilibrium, finding
$$
p = p(\rho, e, Y_{\text{eq}}) + \epsilon \left. \partial_Y p \right|_{\text{eq}} Y_1 = p + \Pi,
$$
where the bulk viscosity $\Pi = - \zeta \theta$, and the bulk viscosity coefficient is
$$
\zeta = -\epsilon \left. \partial_Y p \right|_{\text{eq}} A^{-1} \left( \rho \partial_\rho Y_{\text{eq}} + (e + p_{\text{eq}}) \partial_e Y_{\text{eq}} \right).
$$

The advantages of the bulk viscosity approach is that the stiffness problem (due to the $\epsilon^{-1}$ term is the evolution equation for $Y$) has vanished (as all bulk viscosity terms have non-negative powers of $\epsilon$), and the number of equations of motion has reduced. The complexity of the system has slightly increased, however. As a whole, we expect the bulk viscous addition to be simpler and faster to solve numerically, particularly when $\epsilon \to 0$.

**Note**. An odd, and somewhat concerning, feature here is that (under minimal assumptions about the equilibrium) the bulk viscosity coefficient is negative. This is because the equilibrium pressure is not the true hydrostatic/thermodynamic pressure: that is instead the full $p$. As $\Pi$ turns up on the opposite side of the equality to the true thermodynamic pressure, the sign works in the opposite direction. We may want to re-consider the conventions here.

### Checking the accuracy of the approximation

The calculation above can be checked for formal accuracy as a function of $\epsilon$. 

The leading order case is where the pressure is simply replaced with the equilibrium pressure. This is the "infinitely fast reaction" case. It should be first order accurate as a function of $\epsilon$. That is, given any observable slow variable (such as $e$) at a time $\mathcal{O}(1)$, the difference between the observable computed by the full system including the reaction term and using the full pressure, and computed with the reduced system that does not included the reaction term and evaluates the pressure at equilibrium, should be proportional to $\epsilon$.

The next order case is where the reduced system includes the bulk viscous term. It should be second order accurate as a function of $\epsilon$: the difference to the full system should be proportional to $\epsilon^2$.

We check this numerically by specifying an explicit equation of state, reaction rate, and initial data. We choose
$$
\begin{aligned}
p &= Y \rho e \\
A &= e \\
B &= \frac{\rho}{3} \\
\rho(0) &= 1 \\
e(0) &= 1 \\
\theta &= 1.
\end{aligned}
$$
From this it follows that
$$
\begin{aligned}
Y_{\text{eq}} &= \frac{\rho}{3 e} \\
\partial_\rho Y_{\text{eq}} &= \frac{1}{3 e} \\
\partial_e Y_{\text{eq}} &= -\frac{\rho}{3 e^2} \\
p_{\text{eq}} &= \frac{\rho^2}{3} \\
\left. \partial_Y p \right|_{\text{eq}} &= \rho e \\
\zeta &= -\epsilon \frac{\rho^2}{3 e} \left( 1 - \frac{3e + \rho^2}{3 \rho e} \right).
\end{aligned}
$$

We can now solve the system explicitly in three cases:

1. the full system;
2. the infinitely fast reaction approximation;
3. the bulk viscous approximation.

In [ ]:
theta = 1
rho0 = 1
e0 = 1

def Y_eq(rho, e):
    return rho / (3 * e)

def p(rho, e, Y):
    return Y * rho * e

def p_eq(rho, e):
    return p(rho, e, Y_eq(rho, e))

def zeta(rho, e, epsilon):
    return -epsilon * rho**2 / (3 * e) * (1 - (3 * e + rho**2) / (3 * rho * e))

def Pi(rho, e, epsilon):
    return - zeta(rho, e, epsilon) * theta

def A(rho, e):
    return e

def B(rho, e):
    return rho / 3

def Gamma(rho, e, Y):
    return A(rho, e) * Y - B(rho, e)

def full_rhs(t, z, epsilon):
    """
    In this case the state vector z = (rho, e, Y)^T.
    """
    dzdt = np.zeros_like(z)
    rho, e, Y = z
    dzdt[0] = - rho * theta
    dzdt[1] = - (e + p(rho, e, Y)) / rho * theta
    dzdt[2] = - Gamma(rho, e, Y) / epsilon
    return dzdt

def average_rhs(t, z, epsilon):
    """
    In this case the state vector z = (rho, e)^T as we deal with the reduced system.
    
    To get the leading order approximation, call with epsilon = 0.
    """
    dzdt = np.zeros_like(z)
    rho, e = z
    dzdt[0] = - rho * theta
    dzdt[1] = - (e + p_eq(rho, e) + Pi(rho, e, epsilon)) / rho * theta
    return dzdt

We will evolve up to $t=1$. We will start the system from equilibrium, so in the full case we use $Y(0) = Y_{\text{eq}}(\rho(0), e(0))$.

In [ ]:
t_end = 1
ts = np.linspace(0, t_end, 200)
Y0 = Y_eq(rho0, e0)

We can now check how the error in the internal energy varies as a function of $\epsilon$.

In [ ]:
epsilons = 0.5**np.arange(7, 15)
errors1 = np.zeros_like(epsilons)
errors2 = np.zeros_like(epsilons)
soln_average = solve_ivp(average_rhs, [0, t_end], [rho0, e0], 
                         args=(0,), dense_output=True,
                         rtol=1e-6, atol=1e-8)
for i, epsilon in enumerate(tqdm(epsilons)):
    soln_full = solve_ivp(full_rhs, [0, t_end], [rho0, e0, Y0], 
                          args=(epsilon,), dense_output=True,
                         rtol=1e-6, atol=1e-8)
    soln_bulk = solve_ivp(average_rhs, [0, t_end], [rho0, e0], 
                          args=(epsilon,), dense_output=True,
                         rtol=1e-6, atol=1e-8)
    errors1[i] = abs(soln_full.sol(ts)[1, -1]-soln_average.sol(ts)[1, -1])
    errors2[i] = abs(soln_full.sol(ts)[1, -1]-soln_bulk.sol(ts)[1, -1])

In [ ]:
p1 = np.polyfit(np.log(epsilons), np.log(errors1), 1)
p2 = np.polyfit(np.log(epsilons), np.log(errors2), 1)
fig, axes = plt.subplots(2, 1, figsize=half_width, sharex=True)
axes[0].loglog(epsilons, errors1, 'kx', label=r'Errors, $\infty$ fast')
axes[0].loglog(epsilons, np.exp(p1[1])*epsilons**p1[0], 'b--', label=fr"$\propto \epsilon^{{{p1[0]:.1f}}}$")
# axes[0].set_xlabel(r"$\epsilon$")
axes[0].legend()
axes[1].loglog(epsilons, errors2, 'kx', label='Errors, Bulk')
axes[1].loglog(epsilons, np.exp(p2[1])*epsilons**p2[0], 'b--', label=fr"$\propto \epsilon^{{{p2[0]:.1f}}}$")
axes[1].set_xlabel(r"$\epsilon$")
axes[1].legend()
fig.tight_layout()
if save_plots:
    fig.savefig('rel_no_bl_conv.svg', bbox_inches='tight')

Now we plot the solutions for $\rho, e$ as a function of time. We plot the full evolution of the species fraction to show its behaviour. We also plot the error in $e$, our primary observable, for both approximations.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=full_width, sharex=True)
axes[0, 0].plot(ts, soln_full.sol(ts)[0, :], label="Full")
axes[0, 0].plot(ts, soln_average.sol(ts)[0, :], label=r"$\infty$ fast")
axes[0, 0].plot(ts, soln_bulk.sol(ts)[0, :], label=r"Bulk viscous")
axes[0, 0].set_xlim(0, 1)
axes[0, 0].set_ylabel(r"$\rho$")
axes[0, 0].legend()
axes[0, 1].plot(ts, soln_full.sol(ts)[1, :], label="Full")
axes[0, 1].plot(ts, soln_average.sol(ts)[1, :], label=r"$\infty$ fast")
axes[0, 1].plot(ts, soln_bulk.sol(ts)[1, :], label=r"Bulk viscous")
axes[0, 1].set_xlim(0, 1)
axes[0, 1].set_ylabel(r"$e$")
axes[0, 1].legend()
axes[1, 0].plot(ts, soln_full.sol(ts)[2, :], label="Full")
axes[1, 0].set_xlim(0, 1)
axes[1, 0].set_xlabel(r"$t$")
axes[1, 0].set_ylabel(r"$Y$")
axes[1, 0].legend()
axes[0, 1].legend()
axes[1, 1].plot(ts, soln_full.sol(ts)[1, :] - soln_average.sol(ts)[1, :], label=r"Full - $\infty$ fast")
axes[1, 1].plot(ts, soln_full.sol(ts)[1, :] - soln_bulk.sol(ts)[1, :], label=r"Full - Bulk")
axes[1, 1].set_xlim(0, 1)
axes[1, 1].set_xlabel(r"$t$")
axes[1, 1].set_ylabel(r"Error")
axes[1, 1].legend()
fig.tight_layout()
if save_plots:
    fig.savefig('rel_no_bl.svg', bbox_inches='tight')
plt.show()

So we see the expected result. The bulk viscous approximation is both more accurate (even at "large" $\epsilon$) and converges faster in the stiff limit.

## Boundary layers

In the previous example we started at equilibrium. This might seem reasonable during the inspiral. However, there are many physical and numerical effects that can locally kick the fluid element far from equilibrium. So we need to check the behaviour of the full system far from the equilibrium surface, and see how accurate the approximations are.

Note that we use a pretty large value of $\epsilon$ here for visual clarity.

In [ ]:
Y0 = 1

epsilon = 1e-1
soln_full = solve_ivp(full_rhs, [0, t_end], [rho0, e0, Y0],
                      args=(epsilon,), dense_output=True)
soln_average = solve_ivp(average_rhs, [0, t_end], [rho0, e0],
                         args=(0,), dense_output=True)
soln_bulk = solve_ivp(average_rhs, [0, t_end], [rho0, e0],
                      args=(epsilon,), dense_output=True)

In [ ]:
ts = np.linspace(0, t_end, 200)
fig, axes = plt.subplots(2, 2, figsize=full_width, sharex=True)
axes[0, 0].plot(ts, soln_full.sol(ts)[0, :], label="Full")
axes[0, 0].plot(ts, soln_average.sol(ts)[0, :], label=r"$\infty$ fast")
axes[0, 0].plot(ts, soln_bulk.sol(ts)[0, :], label=r"Bulk viscous")
axes[0, 0].set_xlim(0, 1)
axes[0, 0].set_ylabel(r"$\rho$")
axes[0, 0].legend()
axes[0, 1].plot(ts, soln_full.sol(ts)[1, :], label="Full")
axes[0, 1].plot(ts, soln_average.sol(ts)[1, :], label=r"$\infty$ fast")
axes[0, 1].plot(ts, soln_bulk.sol(ts)[1, :], label=r"Bulk viscous")
axes[0, 1].set_xlim(0, 1)
axes[0, 1].set_ylabel(r"$e$")
axes[0, 1].legend()
axes[1, 0].plot(ts, soln_full.sol(ts)[2, :], label="Full")
axes[1, 0].set_xlim(0, 1)
axes[1, 0].set_xlabel(r"$t$")
axes[1, 0].set_ylabel(r"$Y$")
axes[1, 0].legend()
axes[0, 1].legend()
axes[1, 1].plot(ts, soln_full.sol(ts)[1, :] - soln_average.sol(ts)[1, :], label=r"Full - $\infty$ fast")
axes[1, 1].plot(ts, soln_full.sol(ts)[1, :] - soln_bulk.sol(ts)[1, :], label=r"Full - Bulk")
axes[1, 1].set_xlim(0, 1)
axes[1, 1].set_xlabel(r"$t$")
axes[1, 1].set_ylabel(r"Error")
axes[1, 1].legend()
fig.tight_layout()
if save_plots:
    fig.savefig('rel_bl.svg', bbox_inches='tight')
plt.show()

We immediately see something that looks like a systematic error in the observable $e(1)$, due to the fast "boundary layer" behaviour as the species fraction $Y$ rapidly evolves to the equilibrium surface.

We need to check what happens as the timescale $\epsilon \to 0$.

In [ ]:
epsilons = 0.5**np.arange(7, 15)
errors1 = np.zeros_like(epsilons)
errors2 = np.zeros_like(epsilons)
soln_average = solve_ivp(average_rhs, [0, t_end], [rho0, e0], 
                         args=(0,), dense_output=True,
                         rtol=1e-6, atol=1e-8)
for i, epsilon in enumerate(tqdm(epsilons)):
    soln_full = solve_ivp(full_rhs, [0, t_end], [rho0, e0, Y0], 
                          args=(epsilon,), dense_output=True,
                         rtol=1e-6, atol=1e-8)
    soln_bulk = solve_ivp(average_rhs, [0, t_end], [rho0, e0], 
                          args=(epsilon,), dense_output=True,
                         rtol=1e-6, atol=1e-8)
    errors1[i] = abs(soln_full.sol(ts)[1, -1]-soln_average.sol(ts)[1, -1])
    errors2[i] = abs(soln_full.sol(ts)[1, -1]-soln_bulk.sol(ts)[1, -1])

In [ ]:
p1 = np.polyfit(np.log(epsilons), np.log(errors1), 1)
p2 = np.polyfit(np.log(epsilons), np.log(errors2), 1)
fig, axes = plt.subplots(2, 1, figsize=half_width, sharex=True)
axes[0].loglog(epsilons, errors1, 'kx', label=r'Errors, $\infty$ fast')
axes[0].loglog(epsilons, np.exp(p1[1])*epsilons**p1[0], 'b--', label=fr"$\propto \epsilon^{{{p1[0]:.1f}}}$")
# axes[0].set_xlabel(r"$\epsilon$")
axes[0].legend()
axes[1].loglog(epsilons, errors2, 'kx', label='Errors, Bulk')
axes[1].loglog(epsilons, np.exp(p2[1])*epsilons**p2[0], 'b--', label=fr"$\propto \epsilon^{{{p2[0]:.1f}}}$")
axes[1].set_xlabel(r"$\epsilon$")
axes[1].legend()
fig.tight_layout()
if save_plots:
    fig.savefig('rel_bl_conv.svg', bbox_inches='tight')

We see that the bulk viscous approximation is still (a tiny bit) better (in absolute terms) than the infinitely fast approach, but that the convergence rate is not better. The boundary layer has hidden the advantages of the bulk viscous approach.

## Matched asymptotics

Going back to the original calculation, we can see that the problem here is due to the assumed form of the expansion. We wrote
$$
Y = \sum_{n=0} \epsilon^n Y_n.
$$
and found
$$
Y_0 = \frac{B}{A} = Y_{\text{eq}}(\rho, e).
$$
This is inconsistent with the initial data (in the limit $\epsilon \to 0$) unless we start from equilibrium.

To get around this we need to do a matched asymptotic expansion. We assume that the first calculation, the *outer expansion*, holds everywhere except in a small region of size $\mathcal{O}(\epsilon)$ near $t=0$. We then assume a different form of expansion holds in this region.

The different expansion is constructed by introducing the fast time variable $\tau = t / \epsilon$ and re-writing the equations of motion as
$$
\begin{aligned}
D_{\tau} \rho &= - \epsilon \rho \theta \\
D_{\tau} e &= - \epsilon (e + p(\rho, e, Y)) \theta \\
D_{\tau} Y &= - \Gamma(\rho, e, Y).
\end{aligned}
$$
We again assume the expansion has the form
$$
Y = \sum_{n=0} \epsilon^n Y_n,
$$
but now $Y_n = Y_n(\tau)$. This is our *inner* expansion.

Again we look at the species fraction evolution equation by order in $\epsilon$. This gives
$$
D_{\tau} Y_0 + A Y_0 = B
$$
from which we find
$$
Y_0 = \frac{B}{A} + C_0 e^{-A \tau} = Y_{\text{eq}} + C_0 e^{-A \tau}.
$$
Now we can use the initial data to fix $C_0 = Y(0) - Y_{\text{eq}}(0) = \Delta Y$, and this expansion must be consistent with the initial data.

Next we solve for the higher order terms, finding
$$
Y_n = C_n e^{-A \tau}.
$$
We won't need these later, but they show how a higher-order approximation could be constructed.

We then (again) substitute into the pressure, expanding about equilibrium, to find
$$
p(\rho, e, Y) = p_{\text{eq}} + \Delta Y \, \partial_Y p \, e^{-A \tau} + \mathcal{O}(\Delta Y^2, \epsilon).
$$

From this we can evaluate the internal energy at the time $\tau = T$, meaning $t =  \epsilon T$, for both our inner and outer expansions. First, for the outer expansion, we have
$$
\begin{aligned}
e(t = \epsilon T) &= e(0) + \int_0^{\epsilon T} \text{d}t \, D_t e \\
&= e(0) - \int_0^{\epsilon T} \text{d}t \, (e + p_{\text{eq}} + \Pi) \theta + \mathcal{O}(\epsilon^2) \\
&= e(0) (1 - \epsilon \theta) - \epsilon T p_{\text{eq}} \theta + \mathcal{O}(\epsilon^2).
\end{aligned}
$$
Next, for the inner expansion, we have
$$
\begin{aligned}
e(\tau = T) &= e(0) + \int_0^{T} \text{d} \tau \, D_{\tau} e \\
&= e(0) - \int_0^{T} \text{d} \tau \, \epsilon (e + p_{\text{eq}} + \Delta Y \, \partial_Y p \, e^{-A \tau}) \theta + \mathcal{O}(\epsilon^2) \\
&= e(0) (1 - \epsilon \theta) - \epsilon T p_{\text{eq}} \theta + \epsilon \frac{\Delta Y \, \partial_Y p}{A} \theta \left( 1 - e^{-A T} \right) + \mathcal{O}(\epsilon^2).
\end{aligned}
$$

Therefore, in order to match the inner and outer expansion at some time $\tau = T$, the "initial value" $e(0)$ for the outer expansion has to be modified to
$$
e(0) \to e(0) - \epsilon \frac{\Delta Y \, \partial_Y p}{A} \theta.
$$
The term in the inner expansion proportional to $e^{-A T}$ is dropped as $\tau = \epsilon t$, so at finite $T$ and in the limit $\epsilon \to 0$ this term is exponentially small.

In [ ]:
Y0 = 1

epsilon = 1e-1

def dYp(rho, e, Y):
    return rho * e

e0_modified = e0 - epsilon * theta * (Y0 - Y_eq(rho0, e0)) * dYp(rho0, e0, Y0) / (A(rho0, e0))

soln_full = solve_ivp(full_rhs, [0, t_end], [rho0, e0, Y0],
                      args=(epsilon,), dense_output=True)
soln_average = solve_ivp(average_rhs, [0, t_end], [rho0, e0],
                         args=(0,), dense_output=True)
soln_bulk = solve_ivp(average_rhs, [0, t_end], [rho0, e0_modified],
                      args=(epsilon,), dense_output=True)

In [ ]:
ts = np.linspace(0, t_end, 200)
fig, axes = plt.subplots(2, 2, figsize=full_width, sharex=True)
axes[0, 0].plot(ts, soln_full.sol(ts)[0, :], label="Full")
axes[0, 0].plot(ts, soln_average.sol(ts)[0, :], label=r"$\infty$ fast")
axes[0, 0].plot(ts, soln_bulk.sol(ts)[0, :], label=r"Bulk viscous")
axes[0, 0].set_xlim(0, 1)
axes[0, 0].set_ylabel(r"$\rho$")
axes[0, 0].legend()
axes[0, 1].plot(ts, soln_full.sol(ts)[1, :], label="Full")
axes[0, 1].plot(ts, soln_average.sol(ts)[1, :], label=r"$\infty$ fast")
axes[0, 1].plot(ts, soln_bulk.sol(ts)[1, :], label=r"Bulk viscous")
axes[0, 1].set_xlim(0, 1)
axes[0, 1].set_ylabel(r"$e$")
axes[0, 1].legend()
axes[1, 0].plot(ts, soln_full.sol(ts)[2, :], label="Full")
axes[1, 0].set_xlim(0, 1)
axes[1, 0].set_xlabel(r"$t$")
axes[1, 0].set_ylabel(r"$Y$")
axes[1, 0].legend()
axes[0, 1].legend()
axes[1, 1].plot(ts, soln_full.sol(ts)[1, :] - soln_average.sol(ts)[1, :], label=r"Full - $\infty$ fast")
axes[1, 1].plot(ts, soln_full.sol(ts)[1, :] - soln_bulk.sol(ts)[1, :], label=r"Full - Bulk")
axes[1, 1].set_xlim(0, 1)
axes[1, 1].set_xlabel(r"$t$")
axes[1, 1].set_ylabel(r"Error")
axes[1, 1].legend()
fig.tight_layout()
if save_plots:
    fig.savefig('rel_bl_correction.svg', bbox_inches='tight')
plt.show()

We immediately see a huge improvement for the bulk viscous approximation (which is the only case where the initial data is modified).

We next check convergence with $\epsilon$:

In [ ]:
epsilons = 0.5**np.arange(7, 15)
errors1 = np.zeros_like(epsilons)
errors2 = np.zeros_like(epsilons)
soln_average = solve_ivp(average_rhs, [0, t_end], [rho0, e0], 
                         args=(0,), dense_output=True,
                         rtol=1e-6, atol=1e-8)
for i, epsilon in enumerate(tqdm(epsilons)):
    e0_modified = e0 - epsilon * theta * (Y0 - Y_eq(rho0, e0)) * dYp(rho0, e0, Y0) / (rho0 * A(rho0, e0))
    
    soln_full = solve_ivp(full_rhs, [0, t_end], [rho0, e0, Y0], 
                          args=(epsilon,), dense_output=True,
                         rtol=1e-6, atol=1e-8)
    soln_bulk = solve_ivp(average_rhs, [0, t_end], [rho0, e0_modified], 
                          args=(epsilon,), dense_output=True,
                         rtol=1e-6, atol=1e-8)
    errors1[i] = abs(soln_full.sol(ts)[1, -1]-soln_average.sol(ts)[1, -1])
    errors2[i] = abs(soln_full.sol(ts)[1, -1]-soln_bulk.sol(ts)[1, -1])

In [ ]:
p1 = np.polyfit(np.log(epsilons), np.log(errors1), 1)
p2 = np.polyfit(np.log(epsilons), np.log(errors2), 1)
fig, axes = plt.subplots(2, 1, figsize=half_width)
axes[0].loglog(epsilons, errors1, 'kx', label=r'Errors, $\infty$ fast')
axes[0].loglog(epsilons, np.exp(p1[1])*epsilons**p1[0], 'b--', label=fr"$\propto \epsilon^{{{p1[0]:.1f}}}$")
# axes[0].set_xlabel(r"$\epsilon$")
axes[0].legend()
axes[1].loglog(epsilons, errors2, 'kx', label='Errors, Bulk')
axes[1].loglog(epsilons, np.exp(p2[1])*epsilons**p2[0], 'b--', label=fr"$\propto \epsilon^{{{p2[0]:.1f}}}$")
axes[1].set_xlabel(r"$\epsilon$")
axes[1].legend()
fig.tight_layout()
if save_plots:
    fig.savefig('rel_bl_correction_conv.svg', bbox_inches='tight')

We see immediately that matching to the boundary layer, and hence modifying the internal energy, is necessary in order to capture the second order (in deviations from equilibrium) effects.

## Conclusion

Bulk viscosity driven by nuclear reactions is a second order effect in deviations from (chemical) equilibrium. Models or numerical methods that enforce chemical equilibrium directly, without correcting for the impact on the internal energy, will miss the boundary layer effects, and therefore will be unable to make any statement about the impact of bulk viscosity.

# Extra stuff

#### PDEs

Need to think how this goes for PDEs.

Take Cattaneo written as
$$
\begin{aligned}
  \partial_t T + \partial_x q &= 0 \\
  \partial_t q &= \epsilon^{-1} \left( - q - \kappa \partial_x T \right).
\end{aligned}
$$
The outer expansion gives
$$
q_0 = - \kappa \partial_x T
$$
and
$$
q_1 = - \partial_t q_0 = \kappa \partial_x \left( - \partial_x T \right).
$$
This gives
$$
\partial_t T = \kappa \partial^{(2)}_x T + \epsilon \kappa^2 \partial^{(4)}_x T
$$
as expected (check the signs of the last term, although it is irrelevant to this calculation).

The inner expansion gives the equations of motion as
$$
\begin{aligned}
  \partial_{\tau} T + \epsilon \partial_x q &= 0 \\
  \partial_{\tau} q &=  - q - \kappa \partial_x T.
\end{aligned}
$$
The power series now gives that $T$ is independent of $\tau$ to leading order and hence
$$
  \partial_{\tau} q_0 =  - q_0 - \kappa \partial_x T
$$
integrates directly to
$$
q_0 = C_0 e^{-\tau} - \kappa \partial_x T.
$$
Comparing to the initial data $q(x, 0)$ and writing $\Delta q = q(x, 0) + \kappa \partial_x T(x, 0)$ we get
$$
q_0 = \Delta q \, e^{-\tau} - \kappa \partial_x T.
$$
From this we have
$$
\partial_{\tau} T = \epsilon \left( -e^{-\tau} \partial_x \Delta q + \kappa \partial^{(2)}_x T \right).
$$

The standard matching as above will then give us that
$$
T(x, 0) \to T(x, 0) - \partial_x \Delta q.
$$

I think exactly this form of the calculation (where the matching term comes from integrating a linear first order ODE giving the $e^{-\tau}$ term) will follow whenever the fast equation has a relaxation term that is linear in the fast variable. This holds true for the MIS equations for the bulk and shear case, I believe. The heat equation is less obvious.

#### Constraints

The "real" problem relies on detailed balance of the radiation species vs the other variables. I think the way to model this is to say that, in addition to the slow and fast equations of motion, that there should also be a constraint $h(x, y)$ that has to hold on the fast (or maybe slow?) time. We then add this constraint to the appropriate equation(s) of motion using a Lagrange multiplier. When the expansion in terms of the fast variables is done there should be a correction term which can be matched.